In [ ]:
from functions import sample_uniform_parameters, sample_correlated_parameters, output_from_parameters, output_from_parameters_with_noise, obtain_theta, perform_CNLS_LASSO, obtain_beta_unique, add_random_variables
from pystoned.plot import plot2d
import numpy as np
import pandas as pd

### Sample the true parameters X (iid vs. Correlated), and create Y
### Obtain theta from the results

In [ ]:
x = sample_correlated_parameters(2, 25, 0.5,min_value=10,max_value=20, seed=1)

In [ ]:
x = sample_uniform_parameters(2,25, seed=2)

In [ ]:
y_true = output_from_parameters(x)

In [ ]:
y_hat = output_from_parameters_with_noise(x)

In [ ]:
import warnings
warnings.filterwarnings('ignore')
theta = obtain_theta(x, y_true)

In [ ]:
theta

### Add random input variables

In [ ]:
x = add_random_variables(x, 7, seed=3)

### Perform the SCNLS-LASSO regression

In [ ]:
model_cnls = perform_CNLS_LASSO(x, y_true, eta=0)
beta = model_cnls.get_beta()
alpha = model_cnls.get_alpha()
beta_unique = obtain_beta_unique(x, alpha, beta).T.round(2).rename(columns={0:'Alpha'})
beta_unique.loc['Total',:] = beta_unique.sum(axis=0)
display(beta_unique)